## 0. Import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
import cv2
import time
import random 
import copy
from PIL import Image
import itertools
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, average_precision_score, balanced_accuracy_score, mean_squared_error, mean_absolute_error

## 0. Data paths and parameters

In [ ]:
# data path
root_dir =  ## TO UPDATE ACCORDINGLY
annot_dir = root_dir + 'FileList.csv'
tracings_dir = root_dir + 'VolumeTracings.csv'

# parameters
batch_size = 
num_workers = 
num_epochs = 
seed = 42
CLASS_NAMES = ['Heart Failure', 'No Finding'] # (<ef_cutoff, >ef_cutoff)
ef_cutoff = 50
plotYorN = False

# model
model_selection = 

## 1. Data

In [ ]:
# read csv
df = pd.read_csv(annot_dir)

# histogram
plt.hist(df['EF'],100, label='original')
plt.ylim([0,600])

# data cleaning 
# remove samples that do not have standard frame height (112)
# remove samples that has less than standard number frames (112)
# remove samples with ef_range
df = df.drop(df[df['FrameHeight']!= 112].index)
df = df.drop(df[df['NumberOfFrames'] < 112].index)
df = df.drop(df[(df['EF'] >= (ef_cutoff-5)) & (df['EF'] <= (ef_cutoff+10))].index)
df['Label'] = df['EF']
df = df.drop(columns=['EF','ESV','EDV','FrameHeight','FrameWidth','FPS'])
df['EDES_1'] = 0
df['EDES_2'] = 0
print('Number of videos total: ', len(df))

# data cleaning
print('Number of videos with HF: ', sum(df['Label'] < ef_cutoff))

# read csv
ef_frames = pd.read_csv(tracings_dir)
ef_frames = ef_frames.drop_duplicates(subset=['FileName','Frame'],ignore_index = True)
ef_frames = ef_frames.sort_values(by=['FileName'])

# data cleaning
ef_frames = ef_frames.drop(columns=['X1','Y1','X2','Y2'])
ef_frames = ef_frames.drop_duplicates(ignore_index=True)
ef_frames['FileName']= ef_frames['FileName'].str.split('.avi').str.get(0)

# iterate over patients
ids = sorted(df.index.tolist())
for i in ids:
    curr_frames = list(ef_frames[ef_frames['FileName'].str.contains(df['FileName'][i])]['Frame'])
    df['EDES_1'][i] = curr_frames[0]
    df['EDES_2'][i] = curr_frames[1]
# print(df)
    
# histogram
plt.hist(df['Label'],100, label='reduced')
plt.ylim([0,600])
plt.legend(loc='upper right')
plt.xlabel('EF')
plt.show()

## Train, Val, Test split

In [ ]:
# patient id split
patient_id = sorted(list(set(df['FileName'])))

# TRAIN
train_idx = df[df['Split']=='TRAIN']['FileName']
print('Number of patients in the training set: ', len(train_idx))

# get the train dataframe and sample
df_train = df[df['FileName'].isin(train_idx)]  
train_idx = list(train_idx)
train_label = list(df_train['Label']) 
train_frame_1 = list(df_train['EDES_1']) 
train_frame_2 = list(df_train['EDES_2']) 

# VAL

# TEST

## 2.1. Input

In [ ]:
def set_seeds(seed):
    # back to random seeds
    random.seed(seed)
    np.random.seed(seed)

    # for cuda
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    
def loadvideo(filename: str) -> np.ndarray:
    """Loads a video from a file.
    Args:
        filename (str): filename of video
    Returns:
        A np.ndarray with dimensions (channels=3, frames, height, width). The
        values will be uint8's ranging from 0 to 255.
    Raises:
        FileNotFoundError: Could not find `filename`
        ValueError: An error occurred while reading the video
    """

    if not os.path.exists(filename):
        raise FileNotFoundError(filename)
    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    v = np.zeros((frame_count, frame_height, frame_width, 3), np.uint8)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        v[count, :, :] = frame

    v = v.transpose((3, 0, 1, 2))
    
    capture.release()
    cv2.destroyAllWindows()

    return v     
        
class DatasetGenerator(Dataset):
    
    def __init__ (self, img_list, label_list, frame_list_1, frame_list_2, transform):
    
        self.transform = transform
        self.frame_list_1 = frame_list_1
        self.frame_list_2 = frame_list_2
        self.img_list = img_list
        self.listImageLabels = label_list
        
        imgLabel_cnt = np.zeros(len(CLASS_NAMES))
        
        # iterate over imgs
        for i in range(len(img_list)):
            imgLabel = label_list[i]
            if imgLabel > ef_cutoff:
                imgLabel_cnt = imgLabel_cnt + [0,1]
            else:
                imgLabel_cnt = imgLabel_cnt + [1,0] 
        print(imgLabel_cnt)
    
    def __getitem__(self, index):
        
        # video path
        curr_file = root_dir + 'Videos/' + self.img_list[index] + '.avi'
        
        # load and pad video
        pad = 6
        video = loadvideo(curr_file)
        c, l, h, w = video.shape
        temp = np.zeros((c, l, h + 2 * pad, w + 2 * pad), dtype=video.dtype)
        temp[:, :, pad:-pad, pad:-pad] = video  # pylint: disable=E1130
        i, j = np.random.randint(0, 2 * pad, 2)
        video = temp[:, :, i:(i + h), j:(j + w)]
        
        # get the frame
        imageData = np.zeros((h,w,c))
        tmp = video[:,self.frame_list_1[index],:,:].transpose(1,2,0)
        imageData[:,:,0] = tmp[:,:,0]
        tmp = video[:,round((self.frame_list_1[index]+self.frame_list_2[index])/2),:,:].transpose(1,2,0)
        imageData[:,:,1] = tmp[:,:,0]
        tmp = video[:,self.frame_list_2[index],:,:].transpose(1,2,0)
        imageData[:,:,2] = tmp[:,:,0]
        if plotYorN:
            plt.subplot(1,3,1)
            plt.imshow(imageData[:,:,0])  
            plt.subplot(1,3,2)
            plt.imshow(imageData[:,:,1])  
            plt.subplot(1,3,3)
            plt.imshow(imageData[:,:,2])  
            plt.pause(0.01)
        imageData = Image.fromarray(imageData.astype('uint8'),'RGB')
        if self.transform != None: imageData = self.transform(imageData)    
        image_label = self.listImageLabels[index]
        
        return imageData, image_label
    
    def __len__(self):
        
        return len(self.listImageLabels)    

## Example plot

In [ ]:
fig = plt.figure(figsize=(20, 4))
index = 15
print(train_idx[index])
curr_file = root_dir + 'Videos/' + train_idx[index] + '.avi'
frames = [train_frame_1[index],train_frame_2[index]]
print(frames)
        
# load and pad video
pad = 6
video = loadvideo(curr_file)
c, l, h, w = video.shape
temp = np.zeros((c, l, h + 2 * pad, w + 2 * pad), dtype=video.dtype)
temp[:, :, pad:-pad, pad:-pad] = video  # pylint: disable=E1130
i, j = np.random.randint(0, 2 * pad, 2)
video = temp[:, :, i:(i + h), j:(j + w)]
print(len(range(frames[0]-20,frames[1]+20,4)))
        
# get the frame
cnt = 1
for i in range(frames[0]-20,frames[1]+20,4):
    tmp = video[:,i,:,:].transpose(1,2,0)
    plt.subplot(1, len(range(frames[0]-20,frames[1]+20,4)), cnt)
    plt.imshow(tmp)  
    plt.axis('off')
    plt.title(str(i))
    cnt = cnt + 1
    
plt.subplots_adjust(wspace=0.05, 
                    hspace=0.05)     

## 2.2. Dataloaders

In [ ]:
# gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 
print('Using {} device'.format(device))
set_seeds(seed)

# transform functions
transCrop = 
transResize = 
transformList = []
transformList.append(transforms.Resize(size=transResize))
transformList.append(transforms.ToTensor())
train_transform=transforms.Compose(transformList)

transformList = []
transformList.append(transforms.Resize(transResize))
transformList.append(transforms.ToTensor())
test_transform=transforms.Compose(transformList)

# datasets
image_datasets = {'train': DatasetGenerator(img_list = train_idx, 
                                              label_list = train_label, 
                                              frame_list_1 = train_frame_1,
                                              frame_list_2 = train_frame_2,
                                              transform=train_transform),
                      'val': DatasetGenerator(img_list = val_idx, 
                                              label_list = val_label, 
                                              frame_list_1 = val_frame_1,
                                              frame_list_2 = val_frame_2,
                                              transform=test_transform), 
                      'test': DatasetGenerator(img_list = test_idx, 
                                              label_list = test_label, 
                                              frame_list_1 = test_frame_1,
                                              frame_list_2 = test_frame_2,
                                              transform=test_transform)}

# dataset sizes
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
print(dataset_sizes)

# dataloader
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=batch_size,
                                              shuffle=True, 
                                              num_workers=num_workers)
               for x in ['train', 'val', 'test']}

id = 50
print(train_idx[id], train_label[id])
plt.imshow(image_datasets['train'][id][0].permute(1,2,0))

## 3. Model training

In [ ]:
def compute_accuracy_metrics(preds_vec, labels_vec, class_names):
    # TODO
    
    return roc_auc, average_precision, balanced_acc, f1_acc
    

# training the model
def train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    # best model parameters
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.Inf
    best_acc = 0
    train_acc = []
    val_acc = []
    train_loss = []
    val_loss = []

    # iterate over epochs
    for epoch in range(num_epochs):
        if epoch % 5 == 0:
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            # set model to training mode
            if phase == 'train':
                model.train()  
                
            # set model to evaluate mode    
            else:
                model.eval()   

            # TODO
            
            if phase == 'train':
                scheduler.step()
                
            # get some metrics
            # TODO
            
            # print
            print('{} Loss: {:.4f} MSE: {:.4f} MAE {:.4f} AUROC: {:.4f} Balanced Acc. {:.4f} Avg. Precision: {:.4f}'.format(
                phase, epoch_loss, mse, mae, auroc, balanced_acc, avg_precision))
            
            # save the accuracy and loss
            if phase == 'train':
                train_acc.append(epoch_acc)
                train_loss.append(epoch_loss)
            elif phase == 'val':
                val_acc.append(epoch_acc)
                val_loss.append(epoch_loss)
            
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

    # time    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_acc, train_loss, val_acc, val_loss 

In [ ]:
# back to random seeds
set_seeds(seed)

# model
if model_selection == 'vgg16':
    model_ft = models.vgg16(pretrained=True)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs, 1)
    model_ft.classifier[6].bias.data[0] = 55.6
elif model_selection == 'resnet18':  
    # TODO
    
elif model_selection == 'resnet34':
    # TODO
    
model_ft = model_ft.to(device)

# loss function
criterion = 
  
# observe that all parameters are being optimized
optimizer_ft = 
scheduler_ft = 

# train model
model_ft, train_acc, train_loss, val_acc, val_loss = train_model(dataloaders,
                                                                 model_ft, 
                                                                 criterion, 
                                                                 optimizer_ft, 
                                                                 scheduler_ft,
                                                                 num_epochs=num_epochs)

# plot
plt.subplot(1,2,1)
plt.plot(range(0, num_epochs), train_acc, label = 'Train')
plt.plot(range(0, num_epochs), val_acc, label = 'Val')
plt.xlabel('Epochs')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(0, num_epochs), train_loss, label = 'Train')
plt.plot(range(0, num_epochs), val_loss, label = 'Val')
plt.xlabel('Epochs')
plt.title('Loss')
plt.legend()
plt.show()

## 4. Testing

In [ ]:
preds_vec = np.zeros(dataset_sizes['test'])
labels_vec = np.zeros(dataset_sizes['test'])
probs_mat = np.zeros((dataset_sizes['test'],1))
cnt = 0

# iterate over data
for inputs, labels in dataloaders['test']:
                
    # send inputs and labels to device
    inputs = inputs.to(device)
    labels = labels.to(device).to(torch.float)
    outputs = model_ft(inputs)
    probs_mat[cnt*batch_size:(cnt+1)*batch_size,:] = outputs.cpu().detach().numpy()
    labels_vec[cnt*batch_size:(cnt+1)*batch_size] = labels.cpu().detach().numpy()
    cnt = cnt + 1
                
# get the metrics
mse = mean_squared_error(labels_vec, probs_mat, squared=False)
mae = mean_absolute_error(labels_vec, probs_mat)
auroc, avg_precision, balanced_acc, _ = compute_accuracy_metrics(probs_mat,labels_vec>ef_cutoff,CLASS_NAMES)  
            
# print
print('{} MSE: {:.4f} MAE {:.4f} AUROC: {:.4f} Balanced Acc. {:.4f} Avg. Precision: {:.4f}'.format(
        'Test set: ', mse, mae, auroc, balanced_acc, avg_precision))

## EF plot

In [ ]:
plt.rcParams['figure.figsize'] = [3, 3]
plt.plot(labels_vec,probs_mat,'.k')
plt.xlim([0,85])
plt.ylim([0,85])
plt.plot(np.linspace(0, 85, 10),np.linspace(0, 85, 10),'b')
plt.xlabel('Actual EF')
plt.ylabel('Predicted EF')

## AUC plot

In [ ]:
import sklearn.metrics

fig = plt.figure(figsize=(3, 3))
plt.plot([0, 1], [0, 1], linewidth=1, color="k", linestyle="--")
for thresh in [ef_cutoff]: # [35, 40, 45, 50]:
    fpr, tpr, _ = sklearn.metrics.roc_curve(labels_vec>thresh, probs_mat)
    print(sklearn.metrics.roc_auc_score(labels_vec>thresh, probs_mat))
    plt.plot(fpr, tpr,'k')

    plt.axis([-0.01, 1.01, -0.01, 1.01])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.tight_layout()

## Sample Outputs

In [ ]:
# plot area
plt.rcParams['figure.figsize'] = [10, 10]

# get the images from the dataloader
inputs, labels = next(iter(dataloaders['test']))
rgb_img = inputs.numpy()
labels = np.round(labels.numpy(),2)
inputs = inputs.to(device)
outputs = np.round(model_ft(inputs).cpu().detach().numpy(),2)

# visualization
idx = 16
for i in range(idx):
    plt.subplot(round(idx/4), 4, i+1)
    plt.imshow(rgb_img[i,:,:].transpose(1,2,0))
    plt.title([labels[i],outputs[i][0]])
    plt.axis('off')